In [1]:
from tool import *
# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',35)

In [21]:
with open('D:/Data/SD_raw_data.bin', 'rb') as f: # SD DB 전체를 불러옴
    df = pickle.load(f)

In [22]:
df = df[df['Item No'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find) == 'Quanta'] # Vendor가 Quanta 인 것 만 필터링
df = df[df['Shipment Date'].dt.year == 2023] # 2023년 실적만 필터링

In [24]:
df['Shipment Date'].unique()

array(['2023-01-03T00:00:00.000000000', '2023-01-07T00:00:00.000000000',
       '2023-01-15T00:00:00.000000000', '2023-02-23T00:00:00.000000000',
       '2023-03-01T00:00:00.000000000', '2023-04-27T00:00:00.000000000',
       '2023-05-24T00:00:00.000000000', '2023-06-14T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [23]:
df.groupby(['DOA Number', 'Shipment Date', 'Item No'])['Shipment Quantity'].sum().reset_index()

,DOA Number,Shipment Date,Item No,Shipment Quantity
0,N5V22A08H-391L,2023-01-03,15U40Q-G.AR30K,2
1,N5V22A08H-391L,2023-01-03,15U50P-G.AR3JK,1
2,N5V22A08H-391L,2023-01-03,15U50P-G.AR50ML,1
3,N5V22A08H-391L,2023-02-23,15U50P-G.AP5VLF,5
4,N5V22A08H-391L,2023-02-23,15U50P-K.AP5HL,1
...,...,...,...,...
61,N5V23522H-391L,2023-06-14,15U40R-G.ARTWK,2
62,N5V23522H-391L,2023-06-14,15U50R-S.AP70ML,2
63,N5V23522H-391L,2023-06-14,16T90R-G.AA5CK,10
64,N5V23522H-391L,2023-06-14,16T90R-G.AA76K,2


In [3]:
# 저장할 때만 사용
with open('D:/Data/SD_raw_data.bin', 'wb') as f:
    pickle.dump(df, f)

In [60]:
df = pd.read_excel(get_filename())

In [61]:
df = df[df['Ship Date'].notnull()]

In [62]:
df['Week Name'] = pd.to_datetime(df['Ship Date']).apply(get_weekname)

In [63]:
df['LG Month'] = pd.to_datetime(df['Ship Date']).apply(get_month_from_date)

In [66]:
df.pivot_table('Ship', index='Model', columns='LG Month', aggfunc=sum).fillna(0).sum()

LG Month
2     1390.0
3    15640.0
4    11359.0
5     9034.0
6     6144.0
dtype: float64

In [108]:
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
    df1 = pickle.load(f)

In [109]:
df1 = df1[df1['Ship Year'] == 2023]

In [113]:
df1.pivot_table('Ship', index='Mapping Model.Suffix', columns='Ship Month', aggfunc=sum).sum()

Ship Month
1.0    14426.0
2.0    14645.0
3.0    19493.0
4.0    11379.0
5.0     9034.0
6.0     6144.0
dtype: float64

Mapping Model.Suffix
14T90Q-G.AP79L         7.0
14T90Q-G.AS50ML       30.0
14T90Q-K.AAB9U1       50.0
14T90Q-K.APG5U1       15.0
14T90R-G.AA50K        30.0
                     ...  
16T90R-K.ADB9U1     1810.0
16T90R-K.ADS9U1       40.0
16TD90R-G.AX56K        2.0
16TD90R-G.AX76K       50.0
16TG90Q-G.AA75KN      64.0
Name: Ship, Length: 94, dtype: float64

In [96]:
pd.concat([df[df['LG Month'] == 5].groupby('Model')['Ship'].sum(), df1[df1['Ship Week month'] == '05'].groupby('Mapping Model.Suffix')['Ship'].sum()], axis=1).to_clipboard()

In [97]:
df1[df1['Mapping Model.Suffix'] == '14T90Q-K.APG5U1']

Mapping Model.Suffix Ship To Shipping Cancel    PO  \
PO No.       BL No                                                           
JPCZ22012332 UTOC500577      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   
JPCZ23001049 UTOC500603      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   
             UTOC500622      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   
JPCZ23001794 UTOC500645      14T90Q-K.APG5U1  ENUS_V        Y      N  10.0   
             UTOC500653      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   
JPCZ23002162 UTOC500716      14T90Q-K.APG5U1  ENUS_V        Y      N  15.0   
             UTOC500718      14T90Q-K.APG5U1  ENUS_V        Y      N  20.0   
JPCZ23003261 UTOC500721      14T90Q-K.APG5U1  ENUS_V        Y      N  45.0   
JPCZ23002162 UTOC500724      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   
JPCZ23003261 UTOC500724      14T90Q-K.APG5U1  ENUS_V        Y      N   5.0   

                        OQC Report               OQC Date OQC Result  Ship  \
PO No.       BL No                                                           
JPCZ22012332 UTOC500577  No Report                      -   Approved   5.0   
JPCZ23001049 UTOC500603  Completed  2023-02-25 00:00:00.0   Approved   5.0   
             UTOC500622  Completed  2023-02-25 00:00:00.0   Approved   5.0   
JPCZ23001794 UTOC500645  Completed  2023-04-06 00:00:00.0   Approved  10.0   
             UTOC500653  Completed  2023-04-06 00:00:00.0   Approved   5.0   
JPCZ23002162 UTOC500716  Completed  2023-06-03 00:00:00.0   Approved  15.0   
             UTOC500718  Completed  2023-06-03 00:00:00.0   Approved  20.0   
JPCZ23003261 UTOC500721  Completed  2023-06-06 00:00:00.0   Approved  45.0   
JPCZ23002162 UTOC500724  Completed  2023-06-03 00:00:00.0   Approved   5.0   
JPCZ23003261 UTOC500724  Completed  2023-06-06 00:00:00.0   Approved   5.0   

                                Issued Date        RSD  Ship Date  \
PO No.       BL No                                                  
JPCZ22012332 UTOC500577 2022-12-07 08:47:07 2023-01-08 2023-01-20   
JPCZ23001049 UTOC500603 2023-02-15 05:01:39 2023-03-12 2023-03-01   
             UTOC500622 2023-02-15 05:01:39 2023-03-12 2023-03-21   
JPCZ23001794 UTOC500645 2023-03-15 13:49:30 2023-04-02 2023-04-09   
             UTOC500653 2023-03-15 13:49:30 2023-04-02 2023-04-15   
JPCZ23002162 UTOC500716 2023-03-29 14:22:09 2023-04-23 2023-06-03   
             UTOC500718 2023-03-29 14:22:09 2023-04-23 2023-06-07   
JPCZ23003261 UTOC500721 2023-05-10 14:32:08 2023-06-11 2023-06-07   
JPCZ23002162 UTOC500724 2023-03-29 14:22:09 2023-04-23 2023-06-11   
JPCZ23003261 UTOC500724 2023-05-10 14:32:08 2023-06-11 2023-06-11   

                            BL Status Method Price Term  Unit Price Currency  \
PO No.       BL No                                                             
JPCZ22012332 UTOC500577  CC Requested    AIR        FOB      869.53      USD   
JPCZ23001049 UTOC500603  CC Requested    AIR        FOB      819.53      USD   
             UTOC500622  CC Requested    AIR        FOB      819.53      USD   
JPCZ23001794 UTOC500645  CC Requested    AIR        FOB      819.53      USD   
             UTOC500653  CC Requested    AIR        FOB      819.53      USD   
JPCZ23002162 UTOC500716  CC Requested    AIR        FOB      854.90      USD   
             UTOC500718  CC Requested    AIR        FOB      854.90      USD   
JPCZ23003261 UTOC500721  CC Requested    AIR        FOB      854.90      USD   
JPCZ23002162 UTOC500724  CC Requested    AIR        FOB      854.90      USD   
JPCZ23003261 UTOC500724  CC Requested    AIR        FOB      854.90      USD   

                        Payment Term   PO1 No     SO No. Final Destination  \
PO No.       BL No                                                           
JPCZ22012332 UTOC500577      Z0060FB  1719729  380157604             USSBT   
JPCZ23001049 UTOC500603      Z0060FB  1741186  380160450             USSBT   
             UTOC500622      Z0060FB  1741186  3

## 1. 선적한 SD 정보를 조회하여 가져옴

### 1) 정상 PO 선적의 경우

In [23]:
fn = 'D:/Downloads/' + [n for n in os.listdir('D:/Downloads/') if 'Excel_' in n][-1]
fn

'D:/Downloads/Excel_573712678 (1).xls'

In [14]:
df_add = pd.read_csv(fn, encoding='cp949', sep='\t') # SD inquiry 에서 다운받은 엑셀 불러옴

# 데이터 타입 변경(Unit Price, Amount -> Float 날짜 데이터를 datetime64로)
df_add['Unit Price'] = df_add['Unit Price'].str.replace(',','').astype(float)
df_add['Amount'] = df_add['Amount'].str.replace(',','').astype(float)
col_name = ['Shipment Date', 'ETA Date', 'CC Completion Date', 'Doc Received Date', 'SD Input Date', 'CO Issue Date']
for name in col_name:
    df_add[name] = pd.to_datetime(df_add[name])

In [15]:
df_add = df_add[df_add['Item No'].str.contains(odm_model)].reset_index(drop=True)
df_add

,House BL No,SD Status,Organization,File No,Seq,Container No,Container Type,Supplier,Invoice No,Description,Shipment Quantity,Item No,Currency,Unit Price,Amount,Shipment Method,Shipment Port,Arrival Port,Final Dest.,Vessel Flight,Shipment Date,ETA Date,CC Completion Date,Import Declaration No,Price Terms Code(SD),Import Biz Type,Sub NCV Type,Product1 Level,Product2 Level,HS Code,Line Num,Doc Received Date,Planner,Demand No,Price Terms Code(PO),SD Input Date,CO Issue Flag,CO Issue Date
0,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Luce 14R3(14T90R) Intel Core, LPDDR5, Windows",2,14T90R-G.AA56K,USD,723.29,1446.58,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,1,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
1,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Luce 14R3(14T90R) Intel Core, LPDDR5, Windows",3,14T90R-G.AP56ML,USD,753.19,2259.57,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,2,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
2,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Panther(15U40R) AMD Ryzen, Windows",2,15U40R-G.ARTWK,USD,356.77,713.54,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,3,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
3,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Raphael(15U50R) Intel CPU, Windows, NVidia",2,15U50R-S.AP70ML,USD,617.26,1234.52,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,4,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
4,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Luce 16R3(16T90R) Intel Core, LPDDR5, Windows",10,16T90R-G.AA5CK,USD,798.64,7986.40,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,5,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
5,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Luce 16R3(16T90R) Intel Core, LPDDR5, Windows",2,16T90R-G.AA76K,USD,846.73,1693.46,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,6,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT
6,8.8911E+11,CC Completion,PCZ,TPCZ2311138,1,NaN,NaN,NaN,NaN,"Luce 16R3(16T90R) Intel Core, LPDDR5, Windows",4,16T90R-G.AP7WL,USD,876.81,3507.24,AIR,CNCKG,KRICN,NaN,CX418,2023-06-14,2023-06-16,2023-06-22,4386223111989M,DAP,NCV,Others,PC,PCNT,8.471300e+09,7,NaT,NaN,NaN,NaN,2023-06-19 15:30:00,N,NaT


### 2) DOA 수리 후 환입되는 선적의 경우

In [24]:
df_add = pd.read_csv(fn, encoding='cp949', sep='\t') # SD inquiry 에서 다운받은 엑셀 불러옴

# 데이터 타입 변경(Unit Price, Amount -> Float,날짜 데이터 -> datetime64)
if df_add['Unit Price'].dtype == 'O':
    df_add['Unit Price'] = df_add['Unit Price'].str.replace(',','').astype(float)
if df_add['Amount'].dtype == 'O':
    df_add['Amount'] = df_add['Amount'].str.replace(',','').astype(float)
col_name = ['Shipment Date', 'ETA Date', 'CC Completion Date', 'Doc Received Date', 'SD Input Date', 'CO Issue Date']
for name in col_name:
    df_add[name] = pd.to_datetime(df_add[name])
df_add['Shipment Quantity'] = df_add['Shipment Quantity'].astype(int)
df_add['House BL No'] = df_add['House BL No'].astype(str)
df_add['Line Num'] = df_add['Line Num'].astype(int)
df_add = df_add[df_add['Import Biz Type'] == 'NCV']
df_add = df_add[df_add['Item No'].notnull()] # 모델명이 비어있지 않은 데이터 필터링
# 모델명 정보로 해당하는 Supplier 코드를 산출하여 Supplier 컬럼에 덮어씀
df_add['Supplier'] = df_add['Item No'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find).replace(vendor_code_map)
df_add = df_add[df_add['Supplier'].isin(list(vendor_code_map.values()))] # Supplier 코드가 Quanta, Pegatron 인 것만 뽑아냄
df_add = df_add.set_index(['House BL No', 'File No', 'Line Num'])
df_add

SD Status Organization  Seq  \
House BL No  File No     Line Num                                    
889110042450 TPCZ2311138 1         CC Completion          PCZ    1   
                         2         CC Completion          PCZ    1   
                         3         CC Completion          PCZ    1   
                         4         CC Completion          PCZ    1   
                         5         CC Completion          PCZ    1   
                         6         CC Completion          PCZ    1   
                         7         CC Completion          PCZ    1   

                                   Container No  Container Type  Supplier  \
House BL No  File No     Line Num                                           
889110042450 TPCZ2311138 1                  NaN             NaN  TW008058   
                         2                  NaN             NaN  TW008058   
                         3                  NaN             NaN  TW008058   
                         4                  NaN             NaN  TW008058   
                         5                  NaN             NaN  TW008058   
                         6                  NaN             NaN  TW008058   
                         7                  NaN             NaN  TW008058   

                                   Invoice No  \
House BL No  File No     Line Num               
889110042450 TPCZ2311138 1                NaN   
                         2                NaN   
                         3                NaN   
                         4                NaN   
                         5                NaN   
                         6                NaN   
                         7                NaN   

                                                                     Description  \
House BL No  File No     Line Num                                                  
889110042450 TPCZ2311138 1         Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         2         Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         3                    Panther(15U40R) AMD Ryzen, Windows   
                         4            Raphael(15U50R) Intel CPU, Windows, NVidia   
                         5         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         6         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         7         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   

                                   Shipment Quantity          Item No  \
House BL No  File No     Line Num                                       
889110042450 TPCZ2311138 1                         2   14T90R-G.AA56K   
                         2                         3  14T90R-G.AP56ML   
                         3                         2   15U40R-G.ARTWK   
                         4                         2  15U50R-S.AP70ML   
                         5                        10   16T90R-G.AA5CK   
                         6                         2   16T90R-G.AA76K   
                         7                         4   16T90R-G.AP7WL   

                                  Currency  Unit Price   Amount  \
House BL No  File No     Line Num                                 
889110042450 TPCZ2311138 1             USD      723.29  1446.58   
                         2             USD      753.19  2259.57   
                         3             USD      356.77   713.54   
                         4             USD      617.26  1234.52   
                         5             USD      798.64  7986.40   
                         6             USD      846.73  1693.46   
                         7             USD      876.81  3507.24   

                                  Shipment Method Shipment Port Arrival Port  \
House BL No  File No     Line Num                                              
889110042450 TPCZ2311138 1                    AIR         CNCKG        KRICN   
             

### DOA 넘버 컬럼 추가

In [25]:
df_add['DOA Number'] = 'N5V23522H-391L'

## 3. 기존 DB에 새로운 SD 정보를 추가 Update함

In [26]:
df.shape # update전 기존 DB의 크기 확인

(2538, 36)

In [27]:
df_add.shape # update전 추가할 데이타 크기 확인

(7, 36)

In [28]:
bl_list = df_add.reset_index()['House BL No'].unique().tolist()
bl_list

['889110042450']

In [29]:
updated_bl_list = []
for bl in bl_list:
    if bl not in df.reset_index()['House BL No'].unique():
        updated_bl_list.append(bl)
updated_bl_list

['889110042450']

In [30]:
df_add = df_add[pd.Series(df_add.reset_index()['House BL No'].isin(updated_bl_list).values, index=df_add.index)] # updated_bl_list에 해당하는 것만 sorting
df_add

SD Status Organization  Seq  \
House BL No  File No     Line Num                                    
889110042450 TPCZ2311138 1         CC Completion          PCZ    1   
                         2         CC Completion          PCZ    1   
                         3         CC Completion          PCZ    1   
                         4         CC Completion          PCZ    1   
                         5         CC Completion          PCZ    1   
                         6         CC Completion          PCZ    1   
                         7         CC Completion          PCZ    1   

                                   Container No  Container Type  Supplier  \
House BL No  File No     Line Num                                           
889110042450 TPCZ2311138 1                  NaN             NaN  TW008058   
                         2                  NaN             NaN  TW008058   
                         3                  NaN             NaN  TW008058   
                         4                  NaN             NaN  TW008058   
                         5                  NaN             NaN  TW008058   
                         6                  NaN             NaN  TW008058   
                         7                  NaN             NaN  TW008058   

                                   Invoice No  \
House BL No  File No     Line Num               
889110042450 TPCZ2311138 1                NaN   
                         2                NaN   
                         3                NaN   
                         4                NaN   
                         5                NaN   
                         6                NaN   
                         7                NaN   

                                                                     Description  \
House BL No  File No     Line Num                                                  
889110042450 TPCZ2311138 1         Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         2         Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         3                    Panther(15U40R) AMD Ryzen, Windows   
                         4            Raphael(15U50R) Intel CPU, Windows, NVidia   
                         5         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         6         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         7         Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   

                                   Shipment Quantity          Item No  \
House BL No  File No     Line Num                                       
889110042450 TPCZ2311138 1                         2   14T90R-G.AA56K   
                         2                         3  14T90R-G.AP56ML   
                         3                         2   15U40R-G.ARTWK   
                         4                         2  15U50R-S.AP70ML   
                         5                        10   16T90R-G.AA5CK   
                         6                         2   16T90R-G.AA76K   
                         7                         4   16T90R-G.AP7WL   

                                  Currency  Unit Price   Amount  \
House BL No  File No     Line Num                                 
889110042450 TPCZ2311138 1             USD      723.29  1446.58   
                         2             USD      753.19  2259.57   
                         3             USD      356.77   713.54   
                         4             USD      617.26  1234.52   
                         5             USD      798.64  7986.40   
                         6             USD      846.73  1693.46   
                         7             USD      876.81  3507.24   

                                  Shipment Method Shipment Port Arrival Port  \
House BL No  File No     Line Num                                              
889110042450 TPCZ2311138 1                    AIR         CNCKG        KRICN   
             

In [31]:
for row in df_add.iterrows():
    df.loc[(row[0][0], row[0][1], row[0][2]), :'DOA Number'] = df_add.loc[(row[0][0], row[0][1], row[0][2]), :]

In [32]:
df.shape # 변경된 DB의 크기 확인

(2545, 36)

In [33]:
# 최종 df의 index의 중복이 없는지 확인(결과가 0 이 되어야함)
df.reset_index()[['House BL No', 'File No', 'Line Num']].duplicated().sum()

0

In [34]:
df.tail(10)

SD Status Organization Seq  \
House BL No  File No     Line Num                                   
889110043500 TPCZ2311115 6         Processing CC          PCZ   1   
                         7         Processing CC          PCZ   1   
                         8         Processing CC          PCZ   1   
889110042450 TPCZ2311138 1         CC Completion          PCZ   1   
                         2         CC Completion          PCZ   1   
                         3         CC Completion          PCZ   1   
                         4         CC Completion          PCZ   1   
                         5         CC Completion          PCZ   1   
                         6         CC Completion          PCZ   1   
                         7         CC Completion          PCZ   1   

                                  Container No Container Type  Supplier  \
House BL No  File No     Line Num                                         
889110043500 TPCZ2311115 6                 NaN            NaN  TW008058   
                         7                 NaN            NaN  TW008058   
                         8                 NaN            NaN  TW008058   
889110042450 TPCZ2311138 1                 NaN            NaN  TW008058   
                         2                 NaN            NaN  TW008058   
                         3                 NaN            NaN  TW008058   
                         4                 NaN            NaN  TW008058   
                         5                 NaN            NaN  TW008058   
                         6                 NaN            NaN  TW008058   
                         7                 NaN            NaN  TW008058   

                                  Invoice No  \
House BL No  File No     Line Num              
889110043500 TPCZ2311115 6               NaN   
                         7               NaN   
                         8               NaN   
889110042450 TPCZ2311138 1               NaN   
                         2               NaN   
                         3               NaN   
                         4               NaN   
                         5               NaN   
                         6               NaN   
                         7               NaN   

                                                                         Description  \
House BL No  File No     Line Num                                                      
889110043500 TPCZ2311115 6         Falcon(15U50Q) Intel ADL Platform, Internal Gr...   
                         7         Falcon(15U50Q) Intel ADL Platform, Internal Gr...   
                         8                         Panther(15U40R) AMD Ryzen, Non OS   
889110042450 TPCZ2311138 1             Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         2             Luce 14R3(14T90R) Intel Core, LPDDR5, Windows   
                         3                        Panther(15U40R) AMD Ryzen, Windows   
                         4                Raphael(15U50R) Intel CPU, Windows, NVidia   
                         5             Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         6             Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   
                         7             Luce 16R3(16T90R) Intel Core, LPDDR5, Windows   

                                   Shipment Quantity          Item No  \
House BL No  File No     Line Num                                       
889110043500 TPCZ2311115 6                         1  15U50Q-G.AP50ML   
                         7                         1  15U50Q-G.AR50ML   
                         8                         1  15UD40R-G.AX56K   
889110042450 TPCZ2311138 1                         2   14T90R-G.AA56K   
                         2                         3  14T90R-G.AP56ML   
                         3                         2   15U40R-G.ARTWK   
                         4                         2  15U50R-S.AP70ML   
                  

In [35]:
# 최종 DB 저장
with open('D:/Data/SD_raw_data.bin', 'wb') as f: # 데이타 저장
    pickle.dump(df, f)

## 3. SD 정보에 DOA 환입 건의 DOA 넘버 추가

### 1) DOA 리턴 대비 수리후 재환입 현황 출력

In [15]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

df1 = df1.pivot_table('S/N', index=['Vendor', 'DOA Number', 'OBD'], aggfunc='count').reset_index()
df1 = df1.rename(columns={'S/N':'DOA QTY'})

with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

df2 = df2[df2['Import Biz Type']=='NCV'].pivot_table('Shipment Quantity', index=['DOA Number'], aggfunc=sum).reset_index()
df2= df2.rename(columns={'Shipment Quantity':'Return QTY'})

df3 = df1.merge(df2, how='left')
df3 = df3.fillna(0)
df3.sort_values(['Vendor', 'OBD'])

with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df4 = pickle.load(f)
df4 = df4[df4['DOA Number'].notnull()]

df4 = df4.pivot_table('S/N', index='DOA Number', aggfunc='count').reset_index()
df4 = df4.rename(columns={'S/N':'Inspection QTY'})

df5 = df3.merge(df4, how='left')
df5 = df5.fillna(0)
df5['Balance QTY'] = df5['DOA QTY'] - df5['Inspection QTY']
df5 = df5.convert_dtypes()
df5

,Vendor,DOA Number,OBD,DOA QTY,Return QTY,Inspection QTY,Balance QTY
0,Pegatron,LG20230410,2023-04-15,14,14,14,0
1,Pegatron,LG211006,2021-10-12,35,35,35,0
2,Pegatron,LG211102,2021-11-09,26,26,26,0
3,Pegatron,LG211201,2021-12-09,63,63,63,0
4,Pegatron,LG220304,2022-03-17,51,51,51,0
5,Pegatron,LG220510,2022-05-25,27,27,27,0
6,Pegatron,LG220721,2022-08-01,39,39,39,0
7,Pegatron,LG220818,2022-09-01,562,562,552,10
8,Pegatron,LG221017,2022-10-23,42,42,42,0
9,Pegatron,LG221108,2022-11-13,45,45,45,0


### 2) 수리후 재환입 DOA건의 SD 정보 중 DOA number가 없는 것 현황 출력

In [16]:
with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

In [17]:
c1 = (df2['Import Biz Type'] == 'NCV')
c2 = (df2['DOA Number'].isnull())
c3 = (df2['Shipment Date'] > '2023-5-1')

df2[c1 & c2 & c3].reset_index().pivot_table('Shipment Quantity', index=['Supplier', 'House BL No', 'Shipment Date'], aggfunc=sum)

,,,Shipment Quantity
Supplier,House BL No,Shipment Date,
TW008058,889110043500,2023-05-24,11


### 3) DOA 넘버가 없는 B/L건에 대해 DOA 넘버 입력(B/L 1건에 DOA number 1건이 선적되었을 경우임)

In [23]:
with open('D:/Data/SD_raw_data.bin', 'rb') as f: #SD DB를 불러옴
    df = pickle.load(f)

In [24]:
df.shape

(2530, 36)

In [25]:
doa_num = 'LG20230410'
bl_num = 'W2303845'

In [26]:
df.loc[bl_num, 'Shipment Quantity'].sum()

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_16236\2368164751.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[bl_num, 'Shipment Quantity'].sum()


14

In [27]:
df.loc[bl_num, 'DOA Number'] = doa_num

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_16236\3213437431.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[bl_num, 'DOA Number'] = doa_num


In [28]:
# 최종 DB 저장
with open('D:/Data/SD_raw_data.bin', 'wb') as f: # 데이타 저장
    pickle.dump(df, f)

### 1) 검색 조건 설정 - 특정 BL 1건으로 RMA Number 2건 이상 모델이 입고되었을 때

In [30]:
vendor = ['Quanta']
model = '.'
con = '.'
bl = '889110043500' # 특정 BL 번호로 검색하기 위해
po = '.'
biz_type = 'NCV' # DOA 수리 후 재환입건을 검색하기 위해
doa_num = 'N5V23424H-391L'

In [19]:
with open('D:/Data/SD_raw_data.bin', 'rb') as f: #SD DB를 불러옴
    df = pickle.load(f)
df = df.reset_index() # 조건 필터링을 위해 불러온 SD DB의 index 컬럼들을 해제

In [31]:
df.shape

(2538, 39)

In [32]:
# SD DB 를 필터링하기 위한 각종 조건 setting
cond1 = (df['Supplier'].replace(dict(map(reversed,vendor_code_map.items()))).isin(vendor))
cond2 = (df['Item No'].str.contains(model, na=True))
cond3 = (df['Container No'].str.contains(con, na=True))
cond4 = (df['House BL No'].astype(str).str.contains(bl, na=True))
cond5 = (df['File No'].str.contains(po, na=True))
cond6 = (df['Import Biz Type'] == biz_type)
cond7 = (df['DOA Number'].isnull()) # 'DOA Number' 컬럼에 DOA 넘버가 들어가지 않은 것만 필터링하기 위해

In [33]:
# 위의 조건으로 SD DB 검색한 결과
df1 = df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7].pivot_table('Shipment Quantity', index=['Item No'], aggfunc=sum)
df1

,Shipment Quantity
Item No,
14T90R-G.AA50K,2
14T90R-G.AA56K,2
14TD90R-G.AX56K,2
15U40R-G.ARTWK,1
15U50Q-G.AP50ML,1
15U50Q-G.AR50ML,1
15UD40R-G.AX56K,1


In [34]:
df2 = pd.read_excel(get_filename(), sheet_name=0) # 환입된 SN 리스트 중 특정 DOA 넘버로만 된 엑셀을 불러옴
df2 = df2[df2['DOA Number'] == doa_num]
df3 = df2.pivot_table('S/N', index='Mapping Model.Suffix', aggfunc='count')
df3

,S/N
Mapping Model.Suffix,
14T90R-G.AA50K,2
14T90R-G.AA56K,2
14TD90R-G.AX56K,2
15U40R-G.ARTWK,1
15U50Q-G.AP50ML,1
15U50Q-G.AR50ML,1
15UD40R-G.AX56K,1


In [35]:
df4 = pd.concat([df1, df3], axis=1, join='inner')
df4 = df4.fillna(0)
df4['Bal'] = df4['Shipment Quantity'] - df4['S/N']
df4.loc['sum'] = df4.sum()
df4

,Shipment Quantity,S/N,Bal
14T90R-G.AA50K,2,2,0
14T90R-G.AA56K,2,2,0
14TD90R-G.AX56K,2,2,0
15U40R-G.ARTWK,1,1,0
15U50Q-G.AP50ML,1,1,0
15U50Q-G.AR50ML,1,1,0
15UD40R-G.AX56K,1,1,0
sum,10,10,0


In [36]:
df4 = df4.drop(index='sum')
df4

,Shipment Quantity,S/N,Bal
14T90R-G.AA50K,2,2,0
14T90R-G.AA56K,2,2,0
14TD90R-G.AX56K,2,2,0
15U40R-G.ARTWK,1,1,0
15U50Q-G.AP50ML,1,1,0
15U50Q-G.AR50ML,1,1,0
15UD40R-G.AX56K,1,1,0


In [37]:
# BL에서 특정 DOA Number 건의 모델 수량만큼만 먼저 DOA Number를 구분 기록하는 로직
max_line_num = df.loc[cond4 & cond7, 'Line Num'].max() + 1
for model in df4.index:
    c1 = (df['House BL No'] == bl)
    c2 = (df['DOA Number'].isnull())
    c3 = (df['Item No'] == model)      # 특정 모델만 필터링하기 위한 조건 설정
    ship_qty = df4.loc[model, 'S/N']   # 특정 DOA Number에 해당하는 모델의 입고 수량을 변수에 할당
    bal_qty = df4.loc[model, 'Bal']    # BL내 특정 모델의 전체 입고 수량에서 특정 DOA Number의 입고 수량을 차감한 잔량을 변수에 할당
    if bal_qty == 0:                   # 만약 차감한 수량이 0 이면
        df.loc[c1 & c2 & c3, 'DOA Number'] = doa_num # 해당 BL의 특정 모델의 DOA 넘버를 DOA Number 컬럼에 기록
    elif bal_qty > 0:                  # 만약 잔량이 0 보다 크다면
        df.loc[c1 & c2 & c3, 'Shipment Quantity'] = ship_qty # 해당 BL의 특정 모델 입고 수량을 ship_qty로 바꾸고
        copied_series = df[c1 & c2 & c3].iloc[0]             # 해당 행(Series)을 복사해둠
        copied_series['Shipment Quantity'] = bal_qty       # 복사된 행의 선적 수량을 잔량 수량으로 바꾸고
        max_line_num += 1
        copied_series['Line Num'] = max_line_num # 새롭게 추가할 Line의 Num을 기존 최대값에서 1 증가시켜 부여
        df.loc[c1 & c2 & c3, 'DOA Number'] = doa_num         # 해당 행의 DOA 넘버를 DOA Number 컬럼에 기록
        df = df.append(copied_series, ignore_index=True)   # 복사된 행(잔량값)을 SD DB에 마지막 행에 추가함

In [38]:
df[(df['House BL No']==bl)]

,House BL No,File No,Line Num,SD Status,Organization,Seq,Container No,Container Type,Supplier,Invoice No,Description,Shipment Quantity,Item No,Currency,Unit Price,Amount,Shipment Method,Shipment Port,Arrival Port,Final Dest.,Vessel Flight,Shipment Date,ETA Date,CC Completion Date,Import Declaration No,Price Terms Code(SD),Import Biz Type,Sub NCV Type,Product1 Level,Product2 Level,HS Code,Doc Received Date,Planner,Demand No,Price Terms Code(PO),SD Input Date,CO Issue Flag,CO Issue Date,DOA Number
2530,889110043500,TPCZ2311115,1,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,cobalt 11_G(11TG50Q)(B2G) Intel UHD Graphics +...,1,11TG50Q-E.AC10KN,USD,287.40,287.40,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V230310H-391L
2531,889110043500,TPCZ2311115,2,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Luce 14R3(14T90R) Intel Core, LPDDR5, Windows",2,14T90R-G.AA50K,USD,723.38,1446.76,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2532,889110043500,TPCZ2311115,3,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Luce 14R3(14T90R) Intel Core, LPDDR5, Windows",2,14T90R-G.AA56K,USD,723.29,1446.58,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2533,889110043500,TPCZ2311115,4,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Luce 14R3(14T90R) Intel Core, LPDDR5, NonOS",2,14TD90R-G.AX56K,USD,719.73,1439.46,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2534,889110043500,TPCZ2311115,5,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Panther(15U40R) AMD Ryzen, Windows",1,15U40R-G.ARTWK,USD,356.77,356.77,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2535,889110043500,TPCZ2311115,6,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Falcon(15U50Q) Intel ADL Platform, Internal Gr...",1,15U50Q-G.AP50ML,USD,429.48,429.48,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2536,889110043500,TPCZ2311115,7,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Falcon(15U50Q) Intel ADL Platform, Internal Gr...",1,15U50Q-G.AR50ML,USD,430.24,430.24,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L
2537,889110043500,TPCZ2311115,8,Processing CC,PCZ,1,NaN,NaN,TW008058,NaN,"Panther(15U40R) AMD Ryzen, Non OS",1,15UD40R-G.AX56K,USD,395.80,395.80,AIR,CNCKG,KRICN,NaN,CX416,2023-05-24,2023-05-26,NaT,NaN,DAP,NCV,Others,PC,PCNT,8471300000.0,NaT,NaN,NaN,NaN,2023-05-26 13:30:42,N,NaT,N5V23424H-391L


In [39]:
df.shape

(2538, 39)

In [419]:
df.loc[(df['House BL No']==bl) & (df['Item No']=='15U40Q-G.AR30K')]

,House BL No,File No,Line Num,SD Status,Organization,Seq,Container No,Container Type,Supplier,Invoice No,Description,Shipment Quantity,Item No,Currency,Unit Price,Amount,Shipment Method,Shipment Port,Arrival Port,Final Dest.,Vessel Flight,Shipment Date,ETA Date,CC Completion Date,Import Declaration No,Price Terms Code(SD),Import Biz Type,Sub NCV Type,Product1 Level,Product2 Level,HS Code,Doc Received Date,Planner,Demand No,Price Terms Code(PO),SD Input Date,CO Issue Flag,CO Issue Date,Inspection,Inspection Date,Judge
389,889110028092,TPCZ2209853,9,CC Completion,PCZ,1,<NA>,<NA>,TW008058,NaN,"Paris(15U40Q) AMD Ryzen, 49 Wh, Win10",6,15U40Q-G.AR30K,USD,350.9,20001.3,AIR,CNCKG,KRICN,NaN,CI5998,2022-02-02,2022-02-05 00:00:00,15-FEB-22,4386222500316M,DDP,NCV,Others,PC,PCNT,0.0,NaN,NaN,NaN,<NA>,2022-02-14 11:57:42,N,NaN,N5V21B08H-391,NaT,<NA>
2487,889110028092,TPCZ2209853,45,CC Completion,PCZ,1,<NA>,<NA>,TW008058,NaN,"Paris(15U40Q) AMD Ryzen, 49 Wh, Win10",51,15U40Q-G.AR30K,USD,350.9,20001.3,AIR,CNCKG,KRICN,NaN,CI5998,2022-02-02,2022-02-05 00:00:00,15-FEB-22,4386222500316M,DDP,NCV,Others,PC,PCNT,0.0,NaN,NaN,NaN,<NA>,2022-02-14 11:57:42,N,NaN,DOA,NaT,<NA>


In [40]:
# 잘 수정이 되었는 지 확인
df[df['House BL No']==bl].fillna('None').pivot_table(['Shipment Quantity'], index=['DOA Number', 'Item No'], aggfunc=sum)

Shipment Quantity
DOA Number      Item No                            
N5V230310H-391L 11TG50Q-E.AC10KN                  1
N5V23424H-391L  14T90R-G.AA50K                    2
                14T90R-G.AA56K                    2
                14TD90R-G.AX56K                   2
                15U40R-G.ARTWK                    1
                15U50Q-G.AP50ML                   1
                15U50Q-G.AR50ML                   1
                15UD40R-G.AX56K                   1

In [41]:
df = df.set_index(['House BL No', 'File No', 'Line Num'])

In [42]:
df.shape

(2538, 36)

## 최종 결과를 저장

In [43]:
with open('D:/Data/SD_raw_data.bin', 'wb') as f: # 최종 DB를 저장
    pickle.dump(df, f)